In [3]:
import pandas as pd
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, Bidirectional, GRU 
from keras import Model
from keras import optimizers
from tensorflow import keras
import time

In [4]:
data_full = pd.read_csv('Intrahour Volatility Dataset.csv')

In [5]:
X = data_full[["Return_Squared", "Hourly Volatility"]]
Y = data_full["target"]
data_set = data_full[["Date","Return_Squared", "Hourly Volatility", "target"]]

In [6]:
splitlimit = int(len(data_full)*0.8)
training_features, test_data = data_set[:splitlimit], data_set[splitlimit:]

In [7]:
#Outlier Detection in training_data_features

training_features["hourly_volatility_rolling_median"] = training_features["Hourly Volatility"].rolling(window=41, center=True, min_periods=1).median()
training_features["return_squared_rolling_median"] = training_features["Return_Squared"].rolling(window=41, center=True, min_periods=1).median()
training_features["volatility minus median"] = (training_features["Hourly Volatility"] - training_features["hourly_volatility_rolling_median"]).abs()
training_features["return minus median"] = (training_features["Return_Squared"] - training_features["return_squared_rolling_median"]).abs()
volatility_outliers_removed = training_features[~(training_features['volatility minus median'] > 5 * training_features['volatility minus median'].median())]
both_outliers_removed = volatility_outliers_removed[~(volatility_outliers_removed['return minus median'] > 5 * volatility_outliers_removed['return minus median'].median())]

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_18281/1338683924.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_features["hourly_volatility_rolling_median"] = training_features["Hourly Volatility"].rolling(window=41, center=True, min_periods=1).median()
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_18281/1338683924.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_features["return_squared_rolling_median"] = training_features["Return_Squared"].rolling(window=4

In [8]:
X_cleaned = both_outliers_removed[["Return_Squared", "Hourly Volatility"]]
Y_cleaned = both_outliers_removed["target"]
data_set_cleaned = both_outliers_removed[["Return_Squared", "Hourly Volatility", "target"]]

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
training_data_features_scaled = scaler.fit_transform(X_cleaned)
data_set_scaled = scaler.fit_transform(data_set_cleaned)

In [10]:
#Reconstructing training data
Z = []

backcandles = 15

for j in range(2):
    Z.append([])
    for i in range(backcandles, training_data_features_scaled.shape[0]):
        Z[j].append(training_data_features_scaled[i-backcandles:i, j])
        
Z = np.moveaxis(Z, [0], [2])    
Z, yi = np.array(Z), np.array(data_set_scaled[backcandles-1:, -1])
y_final = np.reshape(yi,(len(yi),1))
y_final = y_final[1:]

In [24]:
lstm_input = Input(shape = (backcandles, 2), name = 'lstm_input')

inputs = LSTM(80, name='first_layer')(lstm_input)

inputs = Dense(1, name='dense_layer')(inputs)

output = Activation('sigmoid', name = 'output')(inputs)

model = Model(inputs = lstm_input, outputs = output)
model.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ["accuracy"])
start_time = time.time()
history = model.fit(x=Z, y=y_final, epochs = 20, validation_data = (Z, y_final))
end_time = time.time()
training_time = end_time - start_time

Epoch 1/20
1349/1349 [==============================] - 5s 4ms/step - loss: 0.6456 - accuracy: 0.6190 - val_loss: 0.6154 - val_accuracy: 0.6550
Epoch 2/20
1349/1349 [==============================] - 5s 4ms/step - loss: 0.6108 - accuracy: 0.6644 - val_loss: 0.6022 - val_accuracy: 0.6755
Epoch 3/20
1349/1349 [==============================] - 5s 4ms/step - loss: 0.6019 - accuracy: 0.6712 - val_loss: 0.5977 - val_accuracy: 0.6767
Epoch 4/20
1349/1349 [==============================] - 5s 4ms/step - loss: 0.6000 - accuracy: 0.6736 - val_loss: 0.5961 - val_accuracy: 0.6780
Epoch 5/20
1349/1349 [==============================] - 5s 4ms/step - loss: 0.5984 - accuracy: 0.6750 - val_loss: 0.5961 - val_accuracy: 0.6774
Epoch 6/20
1349/1349 [==============================] - 5s 4ms/step - loss: 0.5972 - accuracy: 0.6770 - val_loss: 0.5960 - val_accuracy: 0.6793
Epoch 7/20
1349/1349 [==============================] - 5s 4ms/step - loss: 0.5962 - accuracy: 0.6769 - val_loss: 0.5964 - val_accuracy:

In [58]:
model.summary()
# Evaluate the model on the test data
training_accuracy = model.evaluate(Z)

print(training_time)
print(training_accuracy)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_input (InputLayer)      [(None, 15, 2)]           0         
_________________________________________________________________
first_layer (LSTM)           (None, 80)                26560     
_________________________________________________________________
dense_layer (Dense)          (None, 1)                 81        
_________________________________________________________________
output (Activation)          (None, 1)                 0         
Total params: 26,641
Trainable params: 26,641
Non-trainable params: 0
_________________________________________________________________
1349/1349 [==============================] - 0s 177us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
101.37127017974854
[0.0, 0.0]


In [13]:
gru_input = Input(shape = (backcandles, 2), name = 'gru_input')

inputs = GRU(80, name='first_layer')(gru_input)

inputs = Dense(1, name='dense_layer')(inputs)

output = Activation('sigmoid', name = 'output')(inputs)

model2 = Model(inputs = gru_input, outputs = output)
model2.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ["accuracy"])
start_time2 = time.time()
history2 = model2.fit(x=Z, y=y_final, epochs = 20, validation_data = (Z, y_final))
end_time2 = time.time()
training_time2 = end_time2 - start_time2

Epoch 1/20
1349/1349 [==============================] - 5s 3ms/step - loss: 0.6348 - accuracy: 0.6344 - val_loss: 0.6150 - val_accuracy: 0.6645
Epoch 2/20
1349/1349 [==============================] - 4s 3ms/step - loss: 0.6103 - accuracy: 0.6637 - val_loss: 0.6093 - val_accuracy: 0.6630
Epoch 3/20
1349/1349 [==============================] - 4s 3ms/step - loss: 0.6032 - accuracy: 0.6713 - val_loss: 0.5981 - val_accuracy: 0.6761
Epoch 4/20
1349/1349 [==============================] - 4s 3ms/step - loss: 0.6002 - accuracy: 0.6737 - val_loss: 0.5999 - val_accuracy: 0.6742
Epoch 5/20
1349/1349 [==============================] - 5s 3ms/step - loss: 0.5986 - accuracy: 0.6745 - val_loss: 0.5960 - val_accuracy: 0.6792
Epoch 6/20
1349/1349 [==============================] - 4s 3ms/step - loss: 0.5976 - accuracy: 0.6751 - val_loss: 0.5988 - val_accuracy: 0.6777
Epoch 7/20
1349/1349 [==============================] - 4s 3ms/step - loss: 0.5961 - accuracy: 0.6775 - val_loss: 0.5934 - val_accuracy:

In [14]:
model2.summary()
print(training_time2)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_input (InputLayer)       [(None, 15, 2)]           0         
_________________________________________________________________
first_layer (GRU)            (None, 80)                20160     
_________________________________________________________________
dense_layer (Dense)          (None, 1)                 81        
_________________________________________________________________
output (Activation)          (None, 1)                 0         
Total params: 20,241
Trainable params: 20,241
Non-trainable params: 0
_________________________________________________________________
91.04903197288513


In [15]:
X_test = test_data[["Return_Squared", "Hourly Volatility"]]
Y_test = test_data["target"]
test_dataset = test_data[["Return_Squared", "Hourly Volatility", "target"]]

In [16]:
#Scaling test data
test_scaled = scaler.fit_transform(test_dataset)
X_test_scaled = scaler.fit_transform(X_test)

In [25]:
#Reconstructing test data
T = []

backcandles = 15

for j in range(2):
    T.append([])
    for i in range(backcandles, X_test_scaled.shape[0]):
        T[j].append(X_test_scaled[i-backcandles:i, j])
        
T = np.moveaxis(T, [0], [2])
T, yi_test = np.array(T), np.array(test_scaled[backcandles-1:, -1])
y_final_test = np.reshape(yi_test,(len(yi_test),1))
y_final_test = y_final_test[1:]        

In [ ]:
test_predictions1 = model.predict(T)
test_predicted_classes1 = (test_predictions1 > 0.5).astype(int)

In [56]:
#lstm
test_predictions1 = model.predict(T)
test_predicted_classes1 = (test_predictions1 > 0.5).astype(int)
dataframe = pd.DataFrame(y_final_test, columns = ["target"])
dataframe["predicted"] = test_predicted_classes1
cm_lstm = confusion_matrix(dataframe['predicted'], dataframe['target'])
print(cm_lstm)

[[5314 1459]
 [2753 4852]]


In [57]:
#gru
test_predictions2 = model2.predict(T)
test_predicted_classes2 = (test_predictions2 > 0.5).astype(int)
dataframe2 = pd.DataFrame(y_final_test, columns = ["target"])
dataframe2["predicted"] = test_predicted_classes2
cm_gru = confusion_matrix(dataframe2['predicted'], dataframe2['target'])
print(cm_gru)

[[5214 1257]
 [2853 5054]]
